## ALGO TRADE ALGORTHMS
Working with Interactive Brokers API

#### Sources:
- https://stackoverflow.com/questions/65355985/basic-interactive-brokers-api-with-python-example-not-working

- https://blog.devgenius.io/interactive-brokers-api-for-python-d53eb80cc2b9 (Article)
- https://github.com/jayqbe/python_trading_projects/blob/main/IBHistoricalData.py (Code)

- https://medium.com/swlh/structure-and-communicating-with-interactive-brokers-api-python-78ed9dcaccd7

- https://stackoverflow.com/questions/58236021/how-can-i-know-i-retrieved-data-from-ib-api-reqmktdata (OPTIONS working example SOF)



## --- IB_INSYNC RESOIRCES : https://github.com/erdewit/ib_insync/blob/master/notebooks/option_chain.ipynb


In [29]:
from ib_insync import *
util.startLoop()
ib = IB()
ib.connect('127.0.0.1', 7497, clientId = 50, timeout = 11)
spx = Index('SPX', 'CBOE') #Stock = Apple; #Exchange = Smart
ib.qualifyContracts(spx) #create contract
ib.reqMarketDataType(4) #delay proper data - avoid data retrieval timeout issues


API connection failed: TimeoutError()


In [30]:
[ticker] = ib.reqTickers(spx)
print(ticker)
spxValue = ticker.marketPrice()
print(spxValue)
chains = ib.reqSecDefOptParams(spx.symbol, '', spx.secType, spx.conId)

util.df(chains)

Ticker(contract=Index(conId=416904, symbol='SPX', exchange='CBOE', currency='USD', localSymbol='SPX'), time=datetime.datetime(2022, 7, 5, 19, 40, 40, 936351, tzinfo=datetime.timezone.utc), marketDataType=3, bid=-1.0, bidSize=0.0, ask=-1.0, askSize=0.0, last=3820.98, lastSize=1.0, high=3823.19, low=3742.06, close=3825.33)
3825.33


,exchange,underlyingConId,tradingClass,multiplier,expirations,strikes
0,SMART,416904,SPX,100,"[20220714, 20220818, 20220915, 20221020, 20221...","[100.0, 200.0, 300.0, 400.0, 500.0, 600.0, 700..."
1,CBOE,416904,SPXW,100,"[20220705, 20220706, 20220707, 20220708, 20220...","[200.0, 400.0, 600.0, 800.0, 1000.0, 1100.0, 1..."
2,SMART,416904,SPXW,100,"[20220705, 20220706, 20220707, 20220708, 20220...","[200.0, 400.0, 600.0, 800.0, 1000.0, 1100.0, 1..."
3,CBOE,416904,SPX,100,"[20220714, 20220818, 20220915, 20221020, 20221...","[100.0, 200.0, 300.0, 400.0, 500.0, 600.0, 700..."


In [31]:
chain = next(c for c in chains if c.tradingClass == 'SPX' and c.exchange == 'SMART')
chain

OptionChain(exchange='SMART', underlyingConId='416904', tradingClass='SPX', multiplier='100', expirations=['20220714', '20220818', '20220915', '20221020', '20221117', '20221215', '20230119', '20230216', '20230316', '20230420', '20230518', '20230615', '20230720', '20230914', '20231214', '20240620', '20241219', '20251218', '20261217', '20271216'], strikes=[100.0, 200.0, 300.0, 400.0, 500.0, 600.0, 700.0, 800.0, 900.0, 1000.0, 1100.0, 1150.0, 1200.0, 1250.0, 1300.0, 1350.0, 1375.0, 1400.0, 1450.0, 1475.0, 1500.0, 1525.0, 1550.0, 1575.0, 1600.0, 1625.0, 1650.0, 1675.0, 1700.0, 1725.0, 1750.0, 1775.0, 1800.0, 1825.0, 1850.0, 1875.0, 1900.0, 1925.0, 1950.0, 1975.0, 2000.0, 2025.0, 2050.0, 2075.0, 2100.0, 2125.0, 2150.0, 2175.0, 2200.0, 2225.0, 2250.0, 2275.0, 2300.0, 2325.0, 2350.0, 2375.0, 2400.0, 2425.0, 2450.0, 2475.0, 2500.0, 2525.0, 2550.0, 2575.0, 2600.0, 2625.0, 2650.0, 2675.0, 2700.0, 2725.0, 2750.0, 2775.0, 2800.0, 2825.0, 2850.0, 2875.0, 2900.0, 2925.0, 2930.0, 2940.0, 2950.0, 2960

In [33]:
# strikes = [strike for strike in chain.strikes if strike % 5 == 0 and spxValue - 20 < strike < spxValue + 20]
strikes = [strike for strike in chain.strikes if strike % 5 == 0 and spxValue - 50 < strike < spxValue + 50]
expirations = sorted(exp for exp in chain.expirations)
rights = ['P', 'C'] #PUSH and CALL options

contracts = [Option('SPX', expiration, strike, right, 'SMART', tradingClass='SPX')
        for right in rights # for each option type (CALL / PUT)
        for expiration in expirations # for each expiration date
        for strike in strikes] #for each strike price

contracts = ib.qualifyContracts(*contracts)
contracts[0]


Error 200, reqId 4101: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20230316', strike=4215.0, right='P', exchange='SMART', tradingClass='SPX')
Error 200, reqId 4104: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20230316', strike=4230.0, right='P', exchange='SMART', tradingClass='SPX')
Error 200, reqId 4105: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20230316', strike=4235.0, right='P', exchange='SMART', tradingClass='SPX')
Error 200, reqId 4106: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20230316', strike=4240.0, right='P', exchange='SMART', tradingClass='SPX')
Error 200, reqId 4107: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20230

In [ ]:
tickers = ib.reqTickers(*contracts)

tickers[0]
ib.disconnect()

ConnectionError: Not connected